In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import * 
from pyspark.sql import SparkSession

In [0]:
spark = SparkSession.builder.appName("EcommerceDataPipeline").getOrCreate()

In [0]:
# /mnt/delta/tables/bronze/users
usersDF = spark.read.format("delta").load("/mnt/delta/tables/bronze/users")

In [0]:
# Normalize country code to uppercase
usersDF = usersDF.withColumn("countryCode" , upper(col("countryCode")))

In [0]:
# Handling multiple languages elegently with 'expr' and 'case when'

usersDF = usersDF.withColumn("language_full" ,
                           expr("CASE WHEN language = 'EN' THEN 'ENGLISH' " +
                                "WHEN language = 'FR' THEN 'FRENCH' " +
                                "ELSE 'Other' END ")
                           )

In [0]:
# Correcting potential data entry errors in gender column

usersDF = usersDF.withColumn("gender" ,
                              when(col("gender").startswith("M"), "Male")
                              .when(col("gender").startswith("F"), "Female")
                              .otherwise("Other")
)
                             
                             
                             

In [0]:
# Using 'regexp_replace' to clean 'civilitytitle' values

usersDF = usersDF.withColumn("civilitytitle_clean" , 
                                regexp_replace("civilitytitle" , "(Mme|Ms|Mrs)" ,"Ms"))

In [0]:
#Derive new column 'Year_since_last_login' from 'daysSinceLastLogin'

usersDF = usersDF.withColumn("Year_since_last_login" , col("daysSinceLastLogin")/365)

In [0]:
# calculate age of account in years and categorize into 'account_age_group'

usersDF = usersDF.withColumn("account_age_years" , round(col("seniority") / 365 ,2))
usersDF = usersDF.withColumn("account_age_group",
                             when(col("account_age_years") < 1 , "New")
                             .when((col("account_age_years") >=1) & (col("account_age_years") < 3) , "Intermediate")
                             .otherwise("Experienced"))

In [0]:
# Add a column with the current year for comarison 
usersDF = usersDF.withColumn("current_year", year(current_date()))

In [0]:
# Creatively combining strings to forma a unique user descriptor
usersDF = usersDF.withColumn("user_descriptor" , 
                             concat(col("gender") , lit("_"),
                                    col("countrycode"),lit("_"),
                                    expr("substring(civilitytitle_clean,1,3)"),lit("_"),
                                    col("language_full")
                                    
                                    ))

In [0]:
# Flag long title 
usersDF = usersDF.withColumn("flag_long_title" , length(col("civilitytitle_clean")) >10)

In [0]:
usersDF.printSchema()

root
 |-- identifierHash: string (nullable = true)
 |-- type: string (nullable = true)
 |-- country: string (nullable = true)
 |-- language: string (nullable = true)
 |-- socialNbFollowers: string (nullable = true)
 |-- socialNbFollows: string (nullable = true)
 |-- socialProductsLiked: string (nullable = true)
 |-- productsListed: string (nullable = true)
 |-- productsSold: string (nullable = true)
 |-- productsPassRate: string (nullable = true)
 |-- productsWished: string (nullable = true)
 |-- productsBought: string (nullable = true)
 |-- gender: string (nullable = false)
 |-- civilityGenderId: string (nullable = true)
 |-- civilityTitle: string (nullable = true)
 |-- hasAnyApp: string (nullable = true)
 |-- hasAndroidApp: string (nullable = true)
 |-- hasIosApp: string (nullable = true)
 |-- hasProfilePicture: string (nullable = true)
 |-- daysSinceLastLogin: string (nullable = true)
 |-- seniority: string (nullable = true)
 |-- seniorityAsMonths: string (nullable = true)
 |-- seni

In [0]:
usersDF = usersDF.withColumn("hasAnyApp" , col("hasAnyApp").cast("boolean"))
usersDF = usersDF.withColumn("hasAndroidApp" , col("hasAndroidApp").cast("boolean"))
usersDF = usersDF.withColumn("hasIosApp" , col("hasIosApp").cast("boolean"))
usersDF = usersDF.withColumn("hasProfilePicture" , col("hasProfilePicture").cast("boolean"))

usersDF = usersDF.withColumn("socialNbFollowers" , col("socialNbFollowers").cast(IntegerType()))
usersDF = usersDF.withColumn("socialNbFollows" , col("socialNbFollows").cast(IntegerType()))

usersDF = usersDF.withColumn("productsPassRate" , col("productsPassRate").cast(DecimalType(10,2)))
usersDF = usersDF.withColumn("seniorityAsMonths" , col("seniorityAsMonths").cast(DecimalType(10,2)))
usersDF = usersDF.withColumn("seniorityAsYears" , col("seniorityAsYears").cast(DecimalType(10,2)))








In [0]:
usersDF = usersDF.withColumn("daysSinceLastLogin" ,
                                when(col("daysSinceLastLogin").isNotNull() , 
                                     col("daysSinceLastLogin").cast(IntegerType()))
                                .otherwise(0))

In [0]:
usersDF.select("gender").show()

+------+
|gender|
+------+
|Female|
|Female|
|  Male|
|Female|
|  Male|
|Female|
|Female|
|Female|
|Female|
|  Male|
|Female|
|Female|
|Female|
|Female|
|Female|
|Female|
|Female|
|Female|
|Female|
|  Male|
+------+
only showing top 20 rows


In [0]:
usersDF.write.format("delta").mode("overwrite").save("/mnt/delta/tables/silver/users")

In [0]:
# "/mnt/delta/tables/bronze/buyers

buyersDF = spark.read.format("delta").load("/mnt/delta/tables/bronze/buyers")

In [0]:
# Casting Integer Columns

integer_columns = [
    'buyers', 'topbuyers', 'femalebuyers', 'malebuyers',
    'topfemalebuyers', 'topmalebuyers', 'totalproductsbought',
    'totalproductswished', 'totalproductsliked', 'toptotalproductsbought',
    'toptotalproductswished', 'toptotalproductsliked'
]

for column_name in integer_columns:
    buyersDF = buyersDF.withColumn(column_name , col(column_name).cast(IntegerType()))

In [0]:
# Casting Decimal columns
decimal_columns = [
    'topbuyerratio', 'femalebuyersratio', 'topfemalebuyersratio',
    'boughtperwishlistratio', 'boughtperlikeratio', 'topboughtperwishlistratio',
    'topboughtperlikeratio', 'meanproductsbought', 'meanproductswished',
    'meanproductsliked', 'topmeanproductsbought', 'topmeanproductswished',
    'topmeanproductsliked', 'meanofflinedays', 'topmeanofflinedays',
    'meanfollowers', 'meanfollowing', 'topmeanfollowers', 'topmeanfollowing'
]

for column_name in decimal_columns:
    buyersDF = buyersDF.withColumn(column_name , col(column_name).cast(DecimalType(10,2)))

In [0]:
# Normalize country name 
buyersDF = buyersDF.withColumn("country" , initcap(col("country")))

for col_name in integer_columns:
    buyersDF = buyersDF.fillna({col_name:0})

# Calculate the ratio of female to male buyers
buyersDF = buyersDF.withColumn("female_to_make_ratio" ,
                               round(col("femalebuyers")/(col("malebuyers")+1),2)
                               )

# Determine the market potential by comparing wishlist and purchases
buyersDF = buyersDF.withColumn("wishlist_to_purchase_ratio" , 
                               round(col("totalproductswished")/(col("totalproductsbought") + 1),2)
                               )

# Tag countries with highes engagement ratio 
buyersDF = buyersDF.withColumn("high_engagement" ,
                               when(col("boughtperwishlistratio")> 0.5 , True)
                               .otherwise(False))
# Flag market with incresing female buyers participation
buyersDF = buyersDF.withColumn("growing_female_market",
                               when(col("femalebuyersratio") > col("topfemalebuyersratio") , True)
                               .otherwise(False)
                               )
                               
                               

In [0]:
buyersDF.write.format("delta").mode("overwrite").save("/mnt/delta/tables/silver/buyers")

In [0]:
# /mnt/delta/tables/bronze/sellers
sellersDF = spark.read.format("delta").load("/mnt/delta/tables/bronze/sellers")

In [0]:
sellersDF = sellersDF \
    .withColumn("nbsellers", col("nbsellers").cast(IntegerType())) \
    .withColumn("meanproductssold", col("meanproductssold").cast(DecimalType(10, 2))) \
    .withColumn("meanproductslisted", col("meanproductslisted").cast(DecimalType(10, 2))) \
    .withColumn("meansellerpassrate", col("meansellerpassrate").cast(DecimalType(10, 2))) \
    .withColumn("totalproductssold", col("totalproductssold").cast(IntegerType())) \
    .withColumn("totalproductslisted", col("totalproductslisted").cast(IntegerType())) \
    .withColumn("meanproductsbought", col("meanproductsbought").cast(DecimalType(10, 2))) \
    .withColumn("meanproductswished", col("meanproductswished").cast(DecimalType(10, 2))) \
    .withColumn("meanproductsliked", col("meanproductsliked").cast(DecimalType(10, 2))) \
    .withColumn("totalbought", col("totalbought").cast(IntegerType())) \
    .withColumn("totalwished", col("totalwished").cast(IntegerType())) \
    .withColumn("totalproductsliked", col("totalproductsliked").cast(IntegerType())) \
    .withColumn("meanfollowers", col("meanfollowers").cast(DecimalType(10, 2))) \
    .withColumn("meanfollows", col("meanfollows").cast(DecimalType(10, 2))) \
    .withColumn("percentofappusers", col("percentofappusers").cast(DecimalType(10, 2))) \
    .withColumn("percentofiosusers", col("percentofiosusers").cast(DecimalType(10, 2))) \
    .withColumn("meanseniority", col("meanseniority").cast(DecimalType(10, 2)))

In [0]:
# Normalize country name and gender value 
sellersDF = sellersDF.withColumn("country" , initcap(col("country")))\
                    .withColumn("sex" , initcap(col("sex")))

# Add a column to categorize the number of sellers
sellersDF = sellersDF.withColumn("seller_size_category",
                                 when(col("nbsellers")< 500 , "Small")\
                                 .when((col("nbsellers")>=500) & (col("nbsellers")<2000), "Medium")\
                                 .otherwise("Large")
                                 )

# Calculate the mean product listed per seller as an indicator of seller activity 
sellersDF = sellersDF.withColumn("mean_product_listed_per_seller" , 
                               round(col("totalproductslisted") / col("nbsellers"),2))

# Indentify markets with high seller pass rate
sellersDF = sellersDF.withColumn("high_seller_pass_rate" , 
                                 when(col("meansellerpassrate") > 0.75 , "High")\
                                 .otherwise("Normal"))

mean_pass_rate = sellersDF.select(round(avg("meansellerpassrate"),2).alias("avg_pass_rate")).collect()[0]["avg_pass_rate"]

sellersDF = sellersDF.withColumn("meansellerpassrate" , 
                                 when(col("meansellerpassrate").isNull() , mean_pass_rate)\
                                     .otherwise(col("meansellerpassrate")))

                                 
    

In [0]:
sellersDF.write.format("delta").mode("overwrite").save("/mnt/delta/tables/silver/sellers")

In [0]:
# /mnt/delta/tables/bronze/countries

countriesDF = spark.read.format("delta").load("/mnt/delta/tables/bronze/countries")

In [0]:
countriesDF = countriesDF \
    .withColumn("sellers", col("sellers").cast(IntegerType())) \
    .withColumn("topsellers", col("topsellers").cast(IntegerType())) \
    .withColumn("topsellerratio", col("topsellerratio").cast(DecimalType(10, 2))) \
    .withColumn("femalesellersratio", col("femalesellersratio").cast(DecimalType(10, 2))) \
    .withColumn("topfemalesellersratio", col("topfemalesellersratio").cast(DecimalType(10, 2))) \
    .withColumn("femalesellers", col("femalesellers").cast(IntegerType())) \
    .withColumn("malesellers", col("malesellers").cast(IntegerType())) \
    .withColumn("topfemalesellers", col("topfemalesellers").cast(IntegerType())) \
    .withColumn("topmalesellers", col("topmalesellers").cast(IntegerType())) \
    .withColumn("countrysoldratio", col("countrysoldratio").cast(DecimalType(10, 2))) \
    .withColumn("bestsoldratio", col("bestsoldratio").cast(DecimalType(10, 2))) \
    .withColumn("toptotalproductssold", col("toptotalproductssold").cast(IntegerType())) \
    .withColumn("totalproductssold", col("totalproductssold").cast(IntegerType())) \
    .withColumn("toptotalproductslisted", col("toptotalproductslisted").cast(IntegerType())) \
    .withColumn("totalproductslisted", col("totalproductslisted").cast(IntegerType())) \
    .withColumn("topmeanproductssold", col("topmeanproductssold").cast(DecimalType(10, 2))) \
    .withColumn("topmeanproductslisted", col("topmeanproductslisted").cast(DecimalType(10, 2))) \
    .withColumn("meanproductssold", col("meanproductssold").cast(DecimalType(10, 2))) \
    .withColumn("meanproductslisted", col("meanproductslisted").cast(DecimalType(10, 2))) \
    .withColumn("meanofflinedays", col("meanofflinedays").cast(DecimalType(10, 2))) \
    .withColumn("topmeanofflinedays", col("topmeanofflinedays").cast(DecimalType(10, 2))) \
    .withColumn("meanfollowers", col("meanfollowers").cast(DecimalType(10, 2))) \
    .withColumn("meanfollowing", col("meanfollowing").cast(DecimalType(10, 2))) \
    .withColumn("topmeanfollowers", col("topmeanfollowers").cast(DecimalType(10, 2))) \
    .withColumn("topmeanfollowing", col("topmeanfollowing").cast(DecimalType(10, 2)))

In [0]:
# Transformation

countriesDF = countriesDF.withColumn("country" , initcap(col("country")))

# Calculating the ratio of top sellers to total sellers
countriesDF = countriesDF.withColumn("top_seller_ratio" , 
                                     round(col("topsellers") / col("sellers"),2))

# countriesDF countries with high ratio of female sellers 
countriesDF = countriesDF.withColumn("high_female_seller_ratio" , 
                                     when(col("femalesellersratio") > 0.5 , True).otherwise(False))

# Adding a performance indicator based on the sold/listed ratio
countriesDF = countriesDF.withColumn("performane_indicator",
                                     round(col("toptotalproductssold") / (col("toptotalproductslisted") +1),2))


#Flag countries with exceptionally high performance 

countriesDF = countriesDF.withColumn("high_performance" , 
                                     when(col("performane_indicator") > 0.8 , "True").otherwise("False"))


countriesDF = countriesDF.withColumn("activity_level" , 
                                     when(col("meanofflinedays") < 30 , "Higly Active")
                                    .when((col("meanofflinedays") >=30) & (col("meanofflinedays")<60), "Moderately Active")\
                                     .otherwise("Low Activity"))
                                     
                                     
                                     
                                     
                                     
                                     

In [0]:
countriesDF.write.format("delta").mode("overwrite").save("/mnt/delta/tables/silver/countries")